In [1]:
from processing_functions import update_top_n
import xarray as xr

In [2]:
# Open a local dataset, daily temperature for the year 2018, from ERA5 data
temperature2018 = xr.open_dataset("temperature_2018.nc")
temperature2018

<xarray.Dataset> Size: 2GB
Dimensions:         (pressure_level: 1, latitude: 721, longitude: 1440,
                     valid_time: 365)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * valid_time      (valid_time) datetime64[ns] 3kB 2018-01-01 ... 2018-12-31
    number          int64 8B ...
Data variables:
    t               (valid_time, pressure_level, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-05T05:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
# Variable t has the dimensions (valid_time, pressure_level, latitude, longitude) when it should only contain (valid_time, latitude, longitude)
temperature2018.pressure_level.values

array([1000.])

In [4]:
# There is only one pressure level, so we can select this without further preprocessing
temperature2018 = temperature2018.isel(pressure_level = 0)
temperature2018

<xarray.Dataset> Size: 2GB
Dimensions:         (latitude: 721, longitude: 1440, valid_time: 365)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude       (longitude) float64 12kB 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * valid_time      (valid_time) datetime64[ns] 3kB 2018-01-01 ... 2018-12-31
    number          int64 8B ...
Data variables:
    t               (valid_time, latitude, longitude) float32 2GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-05T05:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
# Calculate top 10
results = update_top_n(temperature2018, data_var= "t",time_var="valid_time")

In [6]:
# Lets have a look at the results
results.to_dataframe()

top_t top_valid_time
rank latitude longitude                           
1     90.0    0.00       275.076843     2018-08-06
              0.25       275.076843     2018-08-06
              0.50       275.076843     2018-08-06
              0.75       275.076843     2018-08-06
              1.00       275.076843     2018-08-06
...                             ...            ...
10   -90.0    358.75     269.053650     2018-12-17
              359.00     269.053650     2018-12-17
              359.25     269.053650     2018-12-17
              359.50     269.053650     2018-12-17
              359.75     269.053650     2018-12-17

[10382400 rows x 2 columns]

In [7]:
# They are still in Kelvin, subtract -273.5 to convert to Celsius
results["top_t"] = results.top_t - 273.15

In [8]:
# Check top 10 temperature for Cologne # 50.935173, 6.953101
results.sel(latitude = 50.75, longitude = 6.75).to_dataframe()

,latitude,longitude,top_t,top_valid_time
rank,,,,
1,50.75,6.75,30.486810,2018-08-07
2,50.75,6.75,29.900995,2018-07-27
3,50.75,6.75,29.543207,2018-07-26
4,50.75,6.75,27.663416,2018-08-03
5,50.75,6.75,27.350275,2018-07-25
6,50.75,6.75,27.031915,2018-07-30
7,50.75,6.75,26.717706,2018-07-24
8,50.75,6.75,26.523523,2018-08-06
9,50.75,6.75,26.125909,2018-08-04
